In [44]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (234 kB/s)
Reading package lists... Done


In [45]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [46]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [47]:
# Get the data types of the columns.
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [48]:
# Create a new DataFrame with all the proper columns as a float datatype.

new_df = df.withColumn("price", df["price"].cast('float')).withColumn("bedrooms", df["bedrooms"].cast('float')).withColumn("bathrooms", df["bathrooms"].cast('float')).withColumn("sqft_living", df["sqft_living"].cast('float')).withColumn("sqft_lot", df["sqft_lot"].cast('float')).withColumn("floors", df["floors"].cast('float')).withColumn("waterfront", df["waterfront"].cast('float')).withColumn("view", df["view"].cast('float'))


In [49]:
# Get the data types of the columns.
new_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: float (nullable = true)
 |-- bathrooms: float (nullable = true)
 |-- sqft_living: float (nullable = true)
 |-- sqft_lot: float (nullable = true)
 |-- floors: float (nullable = true)
 |-- waterfront: float (nullable = true)
 |-- view: float (nullable = true)



In [50]:
# 2. Create a temporary view of the DataFrame.

new_df.createOrReplaceTempView('home_sales')

In [51]:
# start_time = time.time()

spark.sql("""SELECT * FROM home_sales LIMIT 10""").show()

# print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923.0|     4.0|      3.0|     3167.0| 11733.0|   2.0|       1.0|76.0|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628.0|     2.0|      2.0|     2235.0| 14384.0|   1.0|       0.0|23.0|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866.0|     2.0|      2.0|     2127.0| 10575.0|   2.0|       0.0| 0.0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895.0|     2.0|      2.0|     1631.0| 11149.0|   2.0|       0.0| 0.0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418.0|     3.0|      2.0|     2249.0| 13878.0|   2.0|       0.0| 4.0|
|5aa00529-0533-46b...|2019-01-30|      2017|218712.0|     2.0|  

In [52]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?


spark.sql("""SELECT YEAR(date) AS YEAR,ROUND(AVG(price),2) AS AVERAGE_PRICE FROM home_sales WHERE bedrooms = 4  GROUP BY YEAR(date) ORDER BY 1""").show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [53]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""SELECT YEAR(date_built) AS YEAR_BUILT,ROUND(AVG(price),2) AS AVERAGE_PRICE FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY YEAR(date_built) ORDER BY 1""").show()


+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [54]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?


spark.sql("""SELECT YEAR(date_built) AS YEAR_BUILT,ROUND(AVG(price),2) AS AVERAGE_PRICE FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_lot >= 2000 GROUP BY YEAR(date_built) ORDER BY 1""").show()


+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2010|    280447.23|
|      2011|    281413.45|
|      2012|    295858.68|
|      2013|    295142.13|
|      2014|    294195.13|
|      2015|    291788.36|
|      2016|     287812.6|
|      2017|    282026.59|
+----------+-------------+



In [65]:


# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql(""" SELECT view,ROUND(AVG(price),2) AS AVERAGE_PRICE FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY 1""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 0.0|    403848.51|
| 1.0|    401044.25|
| 2.0|    397389.25|
| 3.0|     398867.6|
| 4.0|    399631.89|
| 5.0|    401471.82|
| 6.0|    395655.38|
| 7.0|    403005.77|
| 8.0|    398592.71|
| 9.0|    401393.34|
|10.0|    401868.43|
|11.0|    399548.12|
|12.0|    401501.32|
|13.0|    398917.98|
|14.0|    398570.03|
|15.0|     404673.3|
|16.0|    399586.53|
|17.0|    398474.49|
|18.0|    399332.91|
|19.0|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.9607143402099609 seconds ---


In [56]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [57]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [58]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql(""" SELECT view,ROUND(AVG(price),2) AS AVERAGE_PRICE FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY 1""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 0.0|    403848.51|
| 1.0|    401044.25|
| 2.0|    397389.25|
| 3.0|     398867.6|
| 4.0|    399631.89|
| 5.0|    401471.82|
| 6.0|    395655.38|
| 7.0|    403005.77|
| 8.0|    398592.71|
| 9.0|    401393.34|
|10.0|    401868.43|
|11.0|    399548.12|
|12.0|    401501.32|
|13.0|    398917.98|
|14.0|    398570.03|
|15.0|     404673.3|
|16.0|    399586.53|
|17.0|    398474.49|
|18.0|    399332.91|
|19.0|    398953.17|
+----+-------------+
only showing top 20 rows

--- 1.0614216327667236 seconds ---


In [59]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Write out the data in parquet format
new_df.write.parquet('parquet_hs', mode='overwrite')



new_df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_hs")



In [60]:
# 11. Read the parquet formatted data.

df_parquet = spark.read.parquet('parquet_hs')

In [61]:
# 12. Create a temporary table for the parquet data.

df_parquet.createOrReplaceTempView('t_home_sales')

In [62]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql(""" SELECT view,ROUND(AVG(price),2) AS AVERAGE_PRICE FROM t_home_sales WHERE price >= 350000 GROUP BY view ORDER BY 1""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 0.0|    403848.51|
| 1.0|    401044.25|
| 2.0|    397389.25|
| 3.0|     398867.6|
| 4.0|    399631.89|
| 5.0|    401471.82|
| 6.0|    395655.38|
| 7.0|    403005.77|
| 8.0|    398592.71|
| 9.0|    401393.34|
|10.0|    401868.43|
|11.0|    399548.12|
|12.0|    401501.32|
|13.0|    398917.98|
|14.0|    398570.03|
|15.0|     404673.3|
|16.0|    399586.53|
|17.0|    398474.49|
|18.0|    399332.91|
|19.0|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.6742384433746338 seconds ---


In [63]:
# 14. Uncache the home_sales temporary table.

# Remember to uncache the table as soon as you are done.
spark.sql("uncache table home_sales")



DataFrame[]

In [64]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False